## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### Шаг 1. Откройте файл с данными и изучите общую информацию. 

In [96]:
import pandas as pd
data = pd.read_csv('/datasets/data.csv')
display(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


None

### Вывод

Есть пропущенные данные в столбцах days_employed и total_income. Данные типа float64, то есть вещественное число. Следующим шагом заменим данные в этих столбцах на среднее арифметическое каждого столбца.


### Шаг 2. Предобработка данных

### Обработка пропусков

In [97]:
data['days_employed'] = data['days_employed'].abs()#замена отрицательных значений в столбце 'days_employed'
avg_days = data['days_employed'].mean() #Подсчет среднего в столбце days_employed
avg_income = data.groupby('income_type')['total_income'].transform('median') #подсчет медианного значения в total income
#avg_income = data['total_income'].mean() #Подсчет среднего в столбце total_income
data['days_employed'] = data['days_employed'].fillna(value=avg_days) #Замена пропущенных данных на среднее значение в столбце days_employed
data['total_income'] = data['total_income'].fillna(value=avg_income) #Замена пропущенных данных на среднее значение в столбце total_income
display(data.info()) #вывод обновленной информации о таблице.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


None

### Вывод

Мы рассчитали средее значение для каждого столбца с пропущенными данными. После произвели замену пропущенных данных на средние значения. При выводе таблицы можно увидеть, что сейчас пропусков в данных нет, количество значений во всех строках одинаково.
Данные могли появиться в результате ошибки при вводе информации, или же, так как это сведения о людях, их могли отказаться предоставлять. 

### Замена типа данных

In [98]:
data['days_employed'] = data['days_employed'].astype('int') # замена типа данных столбца days_employed из вещественного числа в целое
data['total_income'] = data['total_income'].astype('int') # замена типа данных столбца total_income из вещественного числа в целое
data['children'] = data['children'].replace(-1, 1) #замена отрицательного значения в детях
data['days_employed'] = data['days_employed'].abs()#замена отрицательных значений в столбце 'days_employed'
print(data.info()) # выводим информацию о таблице


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB
None


### Вывод

В таблице присутствовали данные типа float64, что является "вещественным числом". С помощью функции astype() мы преобразовали их в данные типа int64, что является целым числом. 
С помощью функции info() видим, что в таблице остались только количественные данные типа int и текстовые данные типа object

### Обработка дубликатов

In [99]:
data['education'] = data['education'].str.lower() # приводим все значения в столбце "education" к нижнему регистру
display(data.duplicated().sum()) # подсчитываем количество дубликатов
data = data.drop_duplicates().reset_index(drop = True) # удаляем задублированные строки
display(data.info()) #выводим инфо о таблице, чтобы понять, что задублированные строки удалены

71

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21454 entries, 0 to 21453
Data columns (total 12 columns):
children            21454 non-null int64
days_employed       21454 non-null int64
dob_years           21454 non-null int64
education           21454 non-null object
education_id        21454 non-null int64
family_status       21454 non-null object
family_status_id    21454 non-null int64
gender              21454 non-null object
income_type         21454 non-null object
debt                21454 non-null int64
total_income        21454 non-null int64
purpose             21454 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


None

### Вывод

Сначала приводим значения в столбце "education" к нижнему регистру. 
После выполняем поиск дупликатов. Всего в файле была выявлена 71 полностью идентичная строка. 
После при помощи функции drop_duplicates().reset_index(drop = True) удаляем дублирующиеся строки. 

In [100]:
purpose_unique = str(data['purpose'].unique()) #просматриваем уникальные значения в столбце "purpose"
#display(purpose_unique) # просматриваем данные и комментируем строку, чтобы они не мешали в дальнейшем
from pymystem3 import Mystem #импортируем библиотеку лемм
m = Mystem() # задаем переменную для библиотеки
lemmas = m.lemmatize(purpose_unique) #лемматизируем "purpose_unique"
from collections import Counter #вызываем контейнер для подсчета уникальных лемм
display(Counter(lemmas))  #выводим на печать и просматриваем данные, чтобы выявить цели для кредита  



Counter({"['": 1,
         'покупка': 10,
         ' ': 59,
         'жилье': 7,
         "' '": 23,
         'приобретение': 1,
         'автомобиль': 9,
         'дополнительный': 2,
         'образование': 9,
         "'\n": 14,
         " '": 14,
         'сыграть': 1,
         'свадьба': 3,
         'операция': 4,
         'с': 5,
         'на': 4,
         'проведение': 1,
         'для': 2,
         'семья': 1,
         'недвижимость': 10,
         'коммерческий': 2,
         'жилой': 2,
         'строительство': 3,
         'собственный': 1,
         'подержать': 2,
         'свой': 4,
         'со': 1,
         'заниматься': 2,
         'сделка': 2,
         'получение': 3,
         'высокий': 3,
         'профильный': 1,
         'сдача': 1,
         'ремонт': 1,
         "']\n": 1})

### Вывод

Чаще всего кредит берется на приобретение недвижимости, образование, и автомобили. 
Реже на свадьбы.

### Категоризация данных

In [101]:
def kids_group(children): #создаем функцию для разделения заемщиков на две группы: "есть дети" и "нет детей"
    if children < 1:
        return 'бездетные'
    return 'есть дети'
data['children_status'] = data['children'].apply(kids_group) #создаем столбец, куда сохраняем нашу функцию
#display(data.groupby('family_status')['family_status_id'].mean())
#display(data.groupby('total_income',ascending = False))
def salary_group(total_income): # создаем функцию для разделения заемщиков по уровню дохода
    if total_income <= 50000:
        return 'низкий доход'
    if total_income <= 150000:
        return 'средний доход'
    if total_income <= 300000:
        return 'выше среднего'
    return 'богатые'
data['salary_status'] = data['total_income'].apply(salary_group) #создаем столбец куда сохраняем переменную

def age_group(dob_years): #создаем функцию для разделения заемщиков по возрастным категориям
    if dob_years < 18:
        return 'дети'
    if dob_years < 64:
        return 'взрослые'
    return 'пенсионеры'
data['age_group'] = data['dob_years'].apply(age_group) #сохраняем функцию в столбец

def purpose_lemmas(purpose): #задаем функцию, которая разбивает столбец "purpose" на удобные одинаковые цели
    lemmas_row = m.lemmatize(purpose)
    for p in lemmas_row:
        if 'авто' in p:
            return 'автомобиль'
        if 'недвиж' in p:
            return  'недвижимость'
        if 'жиль' in p:
            return 'недвижимость'
        if 'свад' in p:
            return  'свадьба'
        if 'образов' in p:
            return 'образование'
data['purpose_stat']= data['purpose'].apply(purpose_lemmas) # создаем столбец, в который сохраняем действие функции "purpose_lemmas"
def debt_stat(row): #создаем функцию, которая переведет столбец "debt" в текстовое значение, что более удобно
    debt = row['debt']
    if debt == 0:
        return 'not debt'
    if debt == 1:
        return 'debt'
data['debt_stat'] = data.apply(debt_stat, axis = 1) #создаем отдельный столбец для функции
display(data.info()) # выводим инфо о таблице

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21454 entries, 0 to 21453
Data columns (total 17 columns):
children            21454 non-null int64
days_employed       21454 non-null int64
dob_years           21454 non-null int64
education           21454 non-null object
education_id        21454 non-null int64
family_status       21454 non-null object
family_status_id    21454 non-null int64
gender              21454 non-null object
income_type         21454 non-null object
debt                21454 non-null int64
total_income        21454 non-null int64
purpose             21454 non-null object
children_status     21454 non-null object
salary_status       21454 non-null object
age_group           21454 non-null object
purpose_stat        21454 non-null object
debt_stat           21454 non-null object
dtypes: int64(7), object(10)
memory usage: 2.8+ MB


None

### Вывод

Были категорезированы 5 столбцов, с целью обобщить данные. Теперь с ними можно работать и это облегчит получение выводов в дальнейшем.



### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [103]:
children_pivot = data.pivot_table(index = ['children_status'], columns = 'debt_stat', values = 'debt', aggfunc = 'count') # создаем сводную таблицу для понимания, сколько бездетных и детных должников
children_pivot['total'] = children_pivot['debt'] + children_pivot['not debt'] #добавляем столбец "total" общим показателем детных и бездетных заемщиков
children_pivot['debt_otn'] = (children_pivot['debt'] / children_pivot['total']) * 100 #вычисляем долю должников от общего числа заемщиков для каждой категории
display(children_pivot.head(10)) #выводим результат


debt_stat,debt,not debt,total,debt_otn
children_status,,,,
бездетные,1063,13028,14091,7.543822
есть дети,678,6685,7363,9.208203


### Вывод

Большинство заемщиков бездетны (14091). Заемщиков, у которых есть дети, меньше в два раза - 7363. Процент бездетных должников - 7,5% от общего количества заемщиков. 
В то время, как процент должников среди тех, у кого есть 1-2 ребенка - 9,2%.
Поэтому да, зависимость между наличием детей и кредитными отношениями безусловно есть, можно видеть, что те, у кого есть дети, реже решаются взять кредит, не желая возлагать на себя дополнительную финансовую ответственность. Но при этом процент должников с детьми выше, чем процент должников без детей, что так же дает нам понять, что человеку без детей проще справиться с дополнительной финансовой нагрузкой в виде кредита. 

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [108]:
family_pivot = data.pivot_table(index = ['family_status'], columns = 'debt_stat', values = 'debt', aggfunc = 'count') # создаем сводную таблицу для понимания, сколько должников с разным СП
family_pivot['total'] = family_pivot['debt'] + family_pivot['not debt'] #добавляем столбец "total" с общим показателем по семейному положению
family_pivot['debt_otn'] = (family_pivot['debt'] / family_pivot['total']) * 100 #вычисляем долю должников от общего числа заемщиков для каждой категории
display(family_pivot.head(10)) #выводим результат

debt_stat,debt,not debt,total,debt_otn
family_status,,,,
Не женат / не замужем,274,2536,2810,9.750890
в разводе,85,1110,1195,7.112971
вдовец / вдова,63,896,959,6.569343
гражданский брак,388,3763,4151,9.347145
женат / замужем,931,11408,12339,7.545182


### Вывод

Сразу можно увидеть, что больше всего заемщиков среди тех, кто женат/замужем. 
Процент должников от общего количества женатых заемщиков - 7,5%
Процент должников среди холостых - 9,8%
7,1% должников среди разведенных,
6,6% - среди вдовцов
9,4% - среди живущих в гражданском браке. 
Зависимость есть, самый высокий процент должников среди холостых и живущих в гражданском браке. 
Остальные категории близки друг к другу по значениям. 
Сравнивая с предыдущей категорией, я сразу посмотрела статистику по замужеству и наличию детей. 
Среди тех, кто женат/замужем - очень большой процент бездетных, а среди тех, кто холост есть одинокие люди с детьми и половина людей в гражданском браке имеют детей. 
Соответственно, здесь можно провести параллель, что семейной паре без детей во многих случаях проще выплатить кредит, т.к. у них двойной бюджет, из которого проще выкроить средства на ежемесячный платеж, если нет доп.расходов в лице ребенка. 
И наоборот, одинокому человеку может быть немного сложнее управлять финансами, тем более, если у него есть ребенок.



- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [109]:
salary_pivot = data.pivot_table(index = ['salary_status'], columns = 'debt_stat', values = 'debt', aggfunc = 'count') # создаем сводную таблицу для понимания, какой доход у должников
salary_pivot['total'] = salary_pivot['debt'] + salary_pivot['not debt'] #добавляем столбец "total" общий показатель с группировкой по уровню дохода
salary_pivot['debt_otn'] = (salary_pivot['debt'] / salary_pivot['total']) * 100 #вычисляем долю должников от общего числа заемщиков для каждой категории
display(salary_pivot.head(10)) #выводим результат

debt_stat,debt,not debt,total,debt_otn
salary_status,,,,
богатые,106,1377,1483,7.147674
выше среднего,657,7691,8348,7.870149
низкий доход,23,349,372,6.182796
средний доход,955,10296,11251,8.488134


### Вывод

Больше всего заемщиков со средним доходом.
8,5% должников среди людей со средним доходом. 
7,9% должников среди людей с доходом выше среднего.
7% должников среди богатых
6,2% должников среди бедных. 
В целом можно сказать, что особого влияния нет, т.к. разница от категории к категории - 1%,
а выборка людей с низким доходом нерепрезентативна. 
Можно увидеть, что уровень дохода влияет на принятие решения о взятии кредита, т.к. люди с низким доходом понимают, что скорее всего им будет нечем платить, а богатым людям это не нужно. 
Средний класс - основные потребители кредитных продуктов, т.к. знают, что у них есть возможность выплатить кредит, но при этом не всегда есть деньги на единовременный платеж за дорогой товар. 

- Как разные цели кредита влияют на его возврат в срок?

In [110]:
purpose_pivot = data.pivot_table(index = ['purpose_stat'], columns = 'debt_stat', values = 'debt', aggfunc = 'count') # создаем сводную таблицу для понимания целей должников
purpose_pivot['total'] = purpose_pivot['debt'] + purpose_pivot['not debt'] #добавляем столбец "total" общий показатель с группировкой по целям
purpose_pivot['debt_otn'] = (purpose_pivot['debt'] / purpose_pivot['total']) * 100 #вычисляем долю должников от общего числа заемщиков для каждой цели
display(purpose_pivot.head(10)) #выводим результат

debt_stat,debt,not debt,total,debt_otn
purpose_stat,,,,
автомобиль,403,3903,4306,9.359034
недвижимость,782,10029,10811,7.233373
образование,370,3643,4013,9.220035
свадьба,186,2138,2324,8.003442


### Вывод

Больше всего берут кредит на недвижимость. 
9,4% должников среди взявших кредит на авто.
7% должников среди взявших кредит на недвижимость.
9,2% должников среди взявших кредит на учебу.
8% должников среди взявших кредит на свадьбу. 
Разница в пределах 1%, думаю, что цель кредита не влияет на своевременное погашение, кроме кредита на недвижимость. 
Из квартиры могут выселить и забрать в счет ипотеки, возможно поэтому самый низкий процент должников среди тех, кто взял кредит на недвижимость. 

### Шаг 4. Общий вывод

Изучив таблицу, я увидела, что в ней есть пропуски в графах с зарплатой и стажем. 
Пропуски в столбце со стажем я решила заполнить средними значениями по столбцу. 
Пропуски с зп было решено заполнить медианным значением с группировкой по столбцу "должность". 
Данные пропуски могли возникнуть, т.к. кто-то мог не сообщить изначально данные по заработной плате, а после того, как эти данные стали известны, их не внесли в данную статистику. Данные по стажу, насколько я знаю, предоставлять необзательно, к тому же, большинство данных в стаже были внесены со знаком минус, т.е. скорее всего произошел какой-то сбой при сборе данных, или они были некорректно заполнены операторами. 

На возврат задолженности влияют такие факторы, как наличие детей, при этом заемщиков с детьми в два раза больше, чем бездетных, что говорит нам о том, что люди с детьми не хотят брать на себя дополнительные финансовые обязательства. 
Сложности с возвратом кредита в срок испытывают одинокие люди и те, кто живет гражданским браком. 
Причиной этому могут быть трудности с планированием бюджета, к тому же у части одиноких людей есть дети, в то время как большая часть женатых заемщиков бездетны, что безусловно облегчает бремя кредита.
Уровень дохода влияет только на принятие решения о взятии кредита. Люди с низким доходом предпочитают не возлагать на себя дополнительные финансовые риски, людям с очень хорошим достатком в основном это не нужно. 
В основном кредиты берут люди со средним доходом и выше среднего.
На погашение кредита практически не влияют цели, кроме цели "недвижимость". 
Банк имеет право забрать квартиру в счет погашения ипотеки, а для многих ипотечное жилье - единственное. Скорее всего поэтому среди заемщиков с целью "на недвижимость" самый низкий процент должников. 

Рекомендации: 
На основании полученных мной результатов можно увидеть, что на возврат кредита в срок влияет наличие детей: у заемщиков с детьми чаще возникают сложности с погашением кредита в срок. 
Так же влияет и семейное положение, здесь сложности возникают у одиноких людей и людей, живущих гражданским браком, у которых, скорее всего раздельный личный бюджет и личные кредитные обязательства. 
Основные потребители кредитных продуктов - люди со средним достатком. Здесь есть потенциал роста, т.к. еще есть категории с низким и очень высоким доходом, среди них очень мало заемщиков. 
Цели не влияют на погашение кредита в срок, кроме положительного влияния цели "недвижимость" (у нее самый низкий процент должников). Поэтому здесь можно предложить более тщательно проверять заемщиков с иными целями. 